In [2]:
"""
Created on Fri Apr 8 10:29 2022

Prepare reference metrics in NEMO for tuning and evaluation

@author: Clara Burgard
"""

'\nCreated on Fri Apr 8 10:29 2022\n\nPrepare reference metrics in NEMO for tuning and evaluation\n\n@author: Clara Burgard\n'

In [5]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import multimelt.useful_functions as uf
from multimelt.constants import *
import matplotlib as mpl
import glob

In [6]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [7]:
nemo_run = 'OPM018'

if nemo_run == 'OPM006':
    yy_start = 1980
    yy_end = 2018
elif nemo_run == 'OPM021':
    yy_start = 1989
    yy_end = 2018
elif nemo_run == 'OPM016' or nemo_run == 'OPM018':
    yy_start = 1980
    yy_end = 2008

READ IN DATA

In [8]:
inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_boxes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/BOXES/nemo_5km_'+nemo_run+'/'
path_simple = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
path_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
path_boxes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/BOXES/nemo_5km_'+nemo_run+'/'

outputpath_melt = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
outputpath_melt_helen = '../../data/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'

outputpath = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/'
plot_path = '/bettik/burgardc/PLOTS/generic_plots/'

In [9]:
file_isf_orig = xr.open_mfdataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

file_conc = xr.open_mfdataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)#.chunk(chunks={'x': chunk_size, 'y': chunk_size})

In [10]:
isf_mask = (file_isf['ISF_mask'] == file_isf.Nisf)

In [11]:
xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)

only do once

In [13]:
melt_files = list(sorted(glob.glob(inputpath_data+'cavity_melt_*_Ant_stereo.nc')))
ds_melt  = xr.open_mfdataset(melt_files, concat_dim='new_time', combine='nested')#, chunks={'x': chunksize, 'y': chunksize})


In [15]:
ds_melt['time']

<xarray.DataArray 'time' (time: 30)>
array([   0., 1980., 1981., 1982., 1983., 1984., 1985., 1986., 1987., 1988.,
       1989., 1990., 1991., 1992., 1993., 1994., 1995., 1996., 1997., 1998.,
       1999., 2000., 2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008.])
Coordinates:
  * time     (time) float64 0.0 1.98e+03 1.981e+03 ... 2.007e+03 2.008e+03

In [18]:
ds_melt = ds_melt.squeeze('time')
ds_melt = ds_melt.rename({'new_time': 'time'})
ds_melt = ds_melt.assign_coords(time=np.arange(yy_start, yy_end+1))
ds_melt_cut = uf.cut_domain_stereo(ds_melt, map_lim, map_lim)

ValueError: cannot select a dimension to squeeze out which has length greater than one

In [ ]:
ds_melt_cut.to_netcdf(inputpath_data+'cavity_melt_'+str(yy_start)+'-'+str(yy_end)+'_Ant_stereo.nc')

2D m ice per y

In [20]:
ds_melt_cut = xr.open_mfdataset(inputpath_data+'cavity_melt_'+str(yy_start)+'-'+str(yy_end)+'_Ant_stereo.nc')

In [21]:
melt_m_ice_per_y_whole_cell = -1*(ds_melt_cut['melt_cavity']*yearinsec/rho_i) # convert from kg/m2/s
melt_m_ice_per_y = melt_m_ice_per_y_whole_cell/file_conc_cut['isfdraft_conc'] # correct by ice shelf concentration

In [23]:
melt_m_ice_per_y.to_dataset(name='melt_m_ice_per_y').to_netcdf(outputpath_melt_helen+'melt_rates_2D_NEMO.nc')

/home/ockendeh/miniconda3/envs/nnets_py38/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


1D in Gt per y

In [24]:
chunked_mask = isf_mask.chunk({'x': 500, 'y': 500})
file_isf_conc = file_conc['isfdraft_conc'].where(chunked_mask)
melt_m_ice_per_y_avg = melt_m_ice_per_y_whole_cell.where(chunked_mask).mean(['x','y'])

In [25]:
NEMO_melt_rates_1D = xr.Dataset({'melt_m_ice_per_y_avg': (melt_m_ice_per_y_avg.dims, melt_m_ice_per_y_avg.data)}, 
                                coords=melt_m_ice_per_y_avg.coords)

In [26]:
NEMO_melt_rates_1D['melt_Gt_per_y_tot'] = -1*ds_melt_cut['melt_cavity'].where(chunked_mask).sum(['x','y']) * grid_cell_area * yearinsec / 10**12

NEMO_melt_rates_1D['isf_area'] = (isf_mask*file_isf_conc).sum(['x','y'])*grid_cell_area

In [27]:
NEMO_melt_rates_1D.to_netcdf(outputpath_melt_helen+'melt_rates_1D_NEMO_oneFRIS.nc','w')

Time mean 2D

In [28]:
NEMO_melt_rates_2D = xr.open_mfdataset(outputpath_melt_helen+'melt_rates_2D_NEMO.nc')

In [29]:
NEMO_melt_rates_2D_timmean = NEMO_melt_rates_2D.mean('time')
NEMO_melt_rates_2D_timmean.to_netcdf(outputpath_melt_helen+'melt_rates_2D_NEMO_timmean.nc')

Box 1 

In [30]:
grid_cell_area = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area

isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

In [31]:
# Box characteristics
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS.nc')
box_loc_config2 = box_charac_all_2D['box_location'].sel(box_nb_tot=box_charac_all_1D['nD_config'].sel(config=2))

box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=0).drop('Nisf')

file_isf_conc = file_conc['isfdraft_conc']

# Reference 2D 
ref_melt_2D = xr.open_dataset(outputpath_melt_helen+'melt_rates_2D_NEMO.nc')

# Reference (in m/yr)
box1_list = []
ref_box1_myr_list = []
    
for kisf in tqdm(file_isf.Nisf.values):
    isf_conc_isf = uf.choose_isf(file_isf_conc, isf_stack_mask, kisf)
    box_loc_config_stacked = uf.choose_isf(box1, isf_stack_mask, kisf)
    ref_melt_2D_box1_isf = uf.choose_isf(ref_melt_2D['melt_m_ice_per_y'], isf_stack_mask, kisf).where(np.isfinite(box_loc_config_stacked))
    ref_melt_2D_box1_isf_mean = uf.weighted_mean(ref_melt_2D_box1_isf,['mask_coord'], isf_conc_isf)
    ref_box1_myr_list.append(ref_melt_2D_box1_isf_mean)

ref_box1_myr_all = xr.concat(ref_box1_myr_list, dim='Nisf')

ref_box1_myr_all.to_dataset(name='mean_melt_box1_myr').to_netcdf(outputpath_melt_helen+'melt_rates_box1_NEMO_oneFRIS.nc')

  0%|          | 0/35 [00:00<?, ?it/s]